In [21]:
from PIL import Image, ImageDraw, ImageFont
import pandas as pd
import os

def construct_paths(dataset, sub_folder, number):
    #Construct the paths dynamically
    if debug_mode is False:
        base_path = r"C:\Users\sakar\OneDrive\mt-datas\synthetic_data"
        image_path = os.path.join(base_path, f"{dataset}", "images", sub_folder, f"{number}.png")
        label_paths = os.path.join(base_path, f"{dataset}", "labels", sub_folder, f"{number}.txt")
    
    else:
        base_path = r"C:\Users\sakar\OneDrive\mt-datas\synth_validation_data"
        image_path = os.path.join(base_path, f"{dataset}", f"{dataset}_{number}_pic.png")
        label_paths = os.path.join(base_path, f"{dataset}", f"{dataset}_{number}_coco.txt")
    
    return image_path, label_paths
    
# Function to load and display the image with the specified test part and number
def process_image(dataset, sub_folder, number):
   

    # Load and display the image
    if os.path.exists(construct_paths(dataset, sub_folder, number)[0]):
        img = Image.open(construct_paths(dataset, sub_folder, number)[0])
        width, height = img.size  # Get image dimensions
        print(f"Image dimensions: {width}x{height}")
    else:
        raise FileNotFoundError(f"Image file not found at {construct_paths(dataset, sub_folder, number)[0]}")

    # Load the text file as a dataframe without labels
    if os.path.exists(construct_paths(dataset, sub_folder, number)[1]):
        # Assuming the text file has structured data: normalized center_x, center_y, bbox_width, bbox_height, 8 corner values
        df = pd.read_csv(construct_paths(dataset, sub_folder, number)[1], delimiter=' ', header=None)
    else:
        raise FileNotFoundError(f"Label file not found at {construct_paths(dataset, sub_folder, number)[1]}")

    # Create a drawing context on the image
    draw = ImageDraw.Draw(img)

    # You can specify a font here if needed (e.g., arial.ttf), otherwise Pillow will use a default font
    font = ImageFont.load_default()

    # Loop through each row in the DataFrame to get bounding box and corner details
    for index, row in df.iterrows():
        # Access columns by index (1: Center_X, 2: Center_Y, 3: BBox_Width, 4: BBox_Height)
        center_x = row[1] * width
        center_y = row[2] * height
        bbox_width = row[3] * width
        bbox_height = row[4] * height

        # Calculate the top-left and bottom-right coordinates of the bounding box
        top_left_x = center_x - (bbox_width / 2)
        top_left_y = center_y - (bbox_height / 2)
        bottom_right_x = center_x + (bbox_width / 2)
        bottom_right_y = center_y + (bbox_height / 2)

        # Draw the bounding box in red
        draw.rectangle([top_left_x, top_left_y, bottom_right_x, bottom_right_y], outline="red", width=2)

        # Draw the center point in red
        draw.ellipse([center_x - 3, center_y - 3, center_x + 3, center_y + 3], fill="red", outline="red")

        # Access the corner points
        corner_points = [(row[i] * width, row[i + 1] * height) for i in range(5, 28, 3)]

        # Label the corners from 0 to 7 in blue with white text
        for i, point in enumerate(corner_points):
            x, y = point
            # Get text size using textbbox to adjust the background size
            text = str(i)
            bbox = draw.textbbox((x, y), text, font=font)
            text_width = bbox[2] - bbox[0]
            text_height = bbox[3] - bbox[1]

            # Draw a blue rectangle as background for the label
            draw.rectangle([x, y, x + text_width + 6, y + text_height + 4], fill="blue")

            # Draw white text on top of the blue background
            draw.text((x + 3, y + 2), text, fill="white", font=font)

        # Now connect the corners with green lines to form the 3D bounding box
        # Top four points (0, 1, 2, 3) and Bottom four points (4, 5, 6, 7)
        top_square = corner_points[:4]  # First four points
        bottom_square = corner_points[4:]  # Last four points

        # Connect top square (points 0-1, 1-2, 2-3, 3-0)
        # Connect bottom square (points 4-5, 5-6, 6-7, 7-4)
        # Connect corresponding points between the top and bottom squares (0-4, 1-5, 2-6, 3-7)
        for i in range(4):
            draw.line([top_square[i], top_square[(i + 1) % 4]], fill="green", width=2)
            draw.line([bottom_square[i], bottom_square[(i + 1) % 4]], fill="green", width=2)
            draw.line([top_square[i], bottom_square[i]], fill="green", width=2)

    # Show the image with the 3D bounding boxes, center points, and labeled corners
    img.show()

debug_mode = True

dataset = "0_test"
sub_dir = "val"
number = "22"

process_image(dataset,sub_dir, number)


FileNotFoundError: Image file not found at C:\Users\sakar\OneDrive\mt-datas\synth_validation_data\0_test\0_test_0_pic.png